In [11]:
import numpy as np
import pandas as pd
import scipy.stats as ss
import matplotlib.pyplot as plt
import IPython.display as idis
import seaborn as sns
import copy
import scipy.stats as stat
import functools 
%matplotlib inline
pd.options.display.max_rows = 999
pd.options.display.max_columns = 300
import functools
import smogn

from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import mean_absolute_error 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
import xgboost


In [12]:
np.random.seed(1)
#load data
working_patient_data = pd.read_csv('first_vital_patient_data.csv')  
total_los_patient = pd.read_csv('total_los_patients.csv')

print(working_patient_data.shape)
print(working_patient_data.head(10))
# print(working_patient_data.shape)
# print(total_los_patient)
# drop_long_los = functools.reduce(lambda x, y: x + [y] if (total_los_patient.iloc[y,total_los_patient.columns.get_loc('total_los')] <= 10) & (total_los_patient.iloc[y,total_los_patient.columns.get_loc('total_los')] > 20 )  else x +[], [i for i in range(0,total_los_patient.shape[0])] ,[])
drop_long_los = total_los_patient.index[(total_los_patient['total_los'] <= 20) | (total_los_patient['total_los'] > 90)].tolist()
print(len(drop_long_los))
# print(len(drop_long_los))
working_patient_data = working_patient_data.drop(drop_long_los)
total_los_patient = total_los_patient.drop(drop_long_los)
working_patient_data = working_patient_data.reset_index(drop=True)
total_los_patient = total_los_patient.reset_index(drop=True)

# print(total_los_patient)

drop_list = []

for i in range(0,working_patient_data.shape[1]):
    if working_patient_data.iloc[:,i].isnull().sum() / working_patient_data.shape[0] > 0.5:
        drop_list.append(working_patient_data.columns[i])
        
working_patient_data.drop(drop_list,axis=1,inplace=True)

working_patient_data.shape
# print(total_los_patient.head())
# working_patient_data.iloc[1,:]
# total_los_patient.shape
total_los_series = pd.Series(total_los_patient['total_los'])
working_patient_data['total_los'] = total_los_series.values
working_patient_data.fillna(value=0, inplace=True)
working_patient_data.shape

(30495, 111)
   alanine aminotransferase  albumin  albumin ascites  albumin pleural  \
0                      25.0      1.8              NaN              NaN   
1                      24.0      2.8              NaN              NaN   
2                      23.0      3.0              NaN              NaN   
3                      11.0      NaN              NaN              NaN   
4                       NaN      NaN              NaN              NaN   
5                       NaN      NaN              NaN              NaN   
6                      16.0      3.9              NaN              NaN   
7                       NaN      NaN              NaN              NaN   
8                      45.0      4.9              NaN              NaN   
9                       NaN      NaN              NaN              NaN   

   albumin urine  alkaline phosphate  anion gap  asparate aminotransferase  \
0            NaN                73.0       15.0                       69.0   
1            NaN

(3740, 60)

In [13]:
working_patient_data.head()

,alanine aminotransferase,alkaline phosphate,anion gap,asparate aminotransferase,bicarbonate,bilirubin,blood urea nitrogen,calcium,calcium ionized,calcium urine,cardiac index,cholesterol,creatinine,diastolic blood pressure,glascow coma scale total,glucose,heart rate,hematocrit,hemoglobin,lactate,lactic acid,lymphocytes,magnesium,mean blood pressure,mean corpuscular hemoglobin,mean corpuscular hemoglobin concentration,mean corpuscular volume,neutrophils,oxygen saturation,partial pressure of carbon dioxide,partial pressure of oxygen,partial thromboplastin time,peak inspiratory pressure,ph,ph urine,phosphate,phosphorous,platelets,positive end-expiratory pressure set,potassium,potassium serum,pulmonary artery pressure systolic,red blood cell count pleural,red blood cell count urine,respiratory rate,sodium,systemic vascular resistance,systolic blood pressure,temperature,troponin-t,venous pvo2,white blood cell count,male,female,Government_insurance,Medicaid_insurance,Medicare_insurance,Private_insurance,Self_Pay_insurance,total_los
0,0.0,0.0,14.0,0.0,26.0,0.0,19.0,26.0,0.0,9.6,0.00,106.0,0.7,54.0,15.0,126.0,101.0,36.000000,12.5,0.0,0.0,0.0,2.4,73.000000,29.2,34.8,84.0,0.0,97.0,0.0,0.0,0.0,0.0,3.7,3.7,0.0,243.0,0.0,3.8,0.0,0.0,4.30,18.0,0.0,142.0,109.0,36.444422,0.0,0.0,0.000000,12.8,0.00,0,1,0,0,0,1,0,25.5292
1,112.0,127.0,16.0,142.0,20.0,1.3,55.0,20.0,23.0,8.4,0.00,107.0,2.6,52.5,15.0,137.0,102.0,36.000000,12.5,2.8,2.8,0.0,2.4,63.833324,32.6,34.6,94.0,0.0,91.0,43.0,0.0,50.6,0.0,3.2,3.2,0.0,76.0,0.0,4.2,1.7,15.6,3.82,27.0,0.0,139.0,86.5,37.111106,0.0,0.0,0.000000,18.2,7.32,1,0,0,0,1,0,0,25.5485
2,0.0,0.0,12.0,0.0,26.0,0.0,18.0,26.0,27.0,7.9,1.17,104.0,0.4,57.0,11.0,77.0,74.0,28.500000,9.9,1.1,1.1,5.0,1.9,94.000000,27.9,34.6,81.0,59.0,98.0,41.0,145.0,29.5,14.0,2.9,2.9,0.0,177.0,5.0,3.8,1.3,13.7,3.54,12.5,14.0,138.0,135.0,36.666683,670.0,500.0,0.000000,15.9,7.41,1,0,0,0,0,1,0,27.2188
3,9.0,94.0,11.0,17.0,23.0,0.3,17.0,23.0,0.0,7.2,0.00,110.0,0.4,44.0,15.0,95.0,91.0,26.000000,11.2,0.0,0.0,4.0,1.8,58.333302,30.5,35.4,86.0,84.0,99.0,0.0,0.0,34.1,0.0,3.4,3.4,0.0,41.0,0.0,3.9,0.9,11.6,3.68,20.0,0.0,140.0,87.0,37.833317,0.0,0.0,0.000000,1.6,0.00,1,0,0,0,0,1,0,32.9097
4,0.0,0.0,11.0,0.0,22.0,0.0,13.0,22.0,26.0,8.0,1.07,111.0,0.7,63.0,8.0,115.0,96.0,33.900001,12.2,1.8,1.8,0.0,1.9,82.000000,30.9,34.4,90.0,0.0,100.0,44.0,99.0,22.3,26.0,3.0,3.0,14.0,249.0,5.0,3.7,1.0,12.5,3.96,14.0,14.0,140.0,123.0,37.277789,580.0,600.0,108.300003,13.8,7.36,1,0,1,0,0,0,0,28.9542


In [14]:
#balanced data
balance_patient_data = smogn.smoter(data = working_patient_data, y = 'total_los')
balance_patient_data

r_index: 100%|##########| 605/605 [00:01<00:00, 385.08it/s]


,alanine aminotransferase,alkaline phosphate,anion gap,asparate aminotransferase,bicarbonate,bilirubin,blood urea nitrogen,calcium,calcium ionized,calcium urine,cardiac index,cholesterol,creatinine,diastolic blood pressure,glascow coma scale total,glucose,heart rate,hematocrit,hemoglobin,lactate,lactic acid,lymphocytes,magnesium,mean blood pressure,mean corpuscular hemoglobin,mean corpuscular hemoglobin concentration,mean corpuscular volume,neutrophils,oxygen saturation,partial pressure of carbon dioxide,partial pressure of oxygen,partial thromboplastin time,peak inspiratory pressure,ph,ph urine,phosphate,phosphorous,platelets,positive end-expiratory pressure set,potassium,potassium serum,pulmonary artery pressure systolic,red blood cell count pleural,red blood cell count urine,respiratory rate,sodium,systemic vascular resistance,systolic blood pressure,temperature,troponin-t,venous pvo2,white blood cell count,male,female,Government_insurance,Medicaid_insurance,Medicare_insurance,Private_insurance,Self_Pay_insurance,total_los
0,70.285937,58.443586,14.556414,82.225658,19.887171,2.479753,18.120559,15.157648,21.608964,7.205255,1.144359,112.052549,0.694745,52.496135,6.496135,129.700164,101.955181,33.358577,11.310510,1.866924,1.866924,24.676981,2.388717,69.217928,31.127048,36.077434,86.165378,2.058733,97.165378,44.887171,93.472164,36.987944,28.443586,4.960896,4.960896,24.556414,47.976810,14.165378,5.388717,1.511283,15.957960,3.532179,19.639364,24.451316,142.217928,100.774342,38.314950,339.196299,350.0,56.660733,1.627048,7.266917,0,1,0,0,0,0,0,48.493505
1,9.592794,14.885370,22.669214,9.592794,12.669214,0.132314,80.740171,12.669214,3.307860,8.474563,1.167304,103.653930,4.190065,66.653930,15.000000,192.783844,96.608078,24.978384,8.469978,0.264629,0.000000,4.695415,2.363865,93.361354,26.729258,33.896943,78.661572,93.209170,96.676856,5.954148,0.000000,49.517141,0.000000,7.087773,3.407096,0.000000,139.699782,0.000000,4.266921,1.098472,13.286244,3.171681,21.346070,0.000000,134.992358,155.707424,37.793593,0.000000,0.0,44.301965,7.161572,7.306692,0,0,0,0,0,0,0,42.684404
2,35.203628,41.040726,16.837097,33.280639,19.959274,0.381451,24.389115,0.000000,23.877823,8.775565,0.045206,106.366531,1.475565,94.574601,0.000000,120.733063,88.022583,34.437097,11.087782,1.277379,0.126250,0.000000,1.975565,104.981858,30.836653,32.322177,95.755646,0.000000,96.162903,41.592743,0.000000,23.576491,14.285080,3.232581,3.232581,7.366531,268.882264,5.000000,4.304073,1.683710,18.728952,3.601448,16.244354,10.488708,139.162903,121.122177,36.265210,564.411698,500.0,107.647907,11.757016,7.342851,0,0,0,0,0,0,0,47.934328
3,0.000000,0.000000,17.280872,0.000000,23.656174,0.000000,3.687651,23.656174,27.593221,8.109443,1.092809,95.750605,0.303148,66.845035,15.000000,101.501209,98.527848,32.847215,11.281356,1.354722,0.787409,0.000000,1.803148,92.813558,31.912107,34.262470,93.186442,0.000000,98.031477,47.467314,95.125907,13.477963,0.000000,2.215738,2.215738,0.000000,311.874093,1.719128,3.896852,0.412591,4.641645,3.567893,18.280872,0.000000,133.343826,143.343826,36.495009,0.000000,0.0,0.000000,14.290557,7.367506,1,0,0,0,0,1,0,43.060498
4,22.703013,67.992802,11.890962,61.060251,26.139163,3.834151,34.109038,21.090376,5.642762,8.410904,0.353415,117.218075,1.918795,50.860837,4.921088,98.533724,81.375900,33.975607,11.049640,0.415782,0.415782,22.727406,1.759397,66.157824,32.096268,32.454519,99.139163,70.569582,98.812050,9.800586,0.000000,38.343482,0.000000,2.956385,2.956385,0.000000,79.842176,0.000000,4.029699,1.675180,18.636017,3.426356,19.969875,0.000000,151.248201,107.672887,37.029663,0.000000,0.0,60.939749,4.069155,2.182858,0,0,0,0,0,0,0,42.128654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3732,120.000000,89.000000,15.000000,64.000000,26.000000,0.700000,168.000000,0.000000,28.000000,

In [15]:
total_los = np.array(balance_patient_data['total_los'])

In [16]:
balance_patient_data.drop(['total_los'],axis=1,inplace=True)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(balance_patient_data, total_los, test_size=0.2, random_state=1)
print(X_train)

      alanine aminotransferase  alkaline phosphate  anion gap  \
893                   0.000000            0.000000  13.000000   
1188                 12.824802          157.710625  11.940945   
808                   0.000000            0.000000  11.729558   
1064                  6.000000          329.000000  17.000000   
1023                  0.000000            0.000000  11.000000   
...                        ...                 ...        ...   
3017                115.000000          644.000000  13.000000   
905                   0.000000            0.000000  15.709154   
1096                 20.383325          100.429995  12.068095   
235                  86.048793           39.621939  14.860628   
1061                  0.000000            0.000000  16.438234   

      asparate aminotransferase  bicarbonate  bilirubin  blood urea nitrogen  \
893                    0.000000    23.000000   0.000000            50.000000   
1188                  20.295274    22.588582   0.982284    

In [18]:
clf = SVR(kernel='poly', verbose=True)
clf.fit(X_train, y_train)

training_loss = mean_absolute_error(y_train, clf.predict(X_train))
testing_loss = mean_absolute_error(y_test, clf.predict(X_test))

print('Training loss is {}'.format(training_loss))
print('Testing loss is {}'.format(testing_loss))

[LibSVM]Training loss is 12.231612548316223
Testing loss is 12.393861721092076
